## Import and Configure Everything We Need

In [4]:
%matplotlib inline

from collections import defaultdict as ddict, OrderedDict as odict
from typing import Any, Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from rdkit.Chem import PandasTools, AllChem as Chem, Descriptors
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
import deepchem as dc
import torch

from rdkit import RDLogger

RDLogger.DisableLog('rdApp.*') 

import basic as b
import chemprop_ish as c

pd.set_option('display.float_format', lambda x: '%.3f' % x)  # Display floats without scientific notation

# In many cases NaN
not_used_desc = ['MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge']

# Create a descriptor calculator for all RDKit descriptors except the ones above
desc_calc = MolecularDescriptorCalculator([x for x in [x[0] for x in Descriptors.descList] if x not in not_used_desc])

---
## Loading Precombined Dataset

In [5]:
data = pd.read_csv('combisolv_exp2.csv')
solute = data['smiles_solute'].tolist()
solvent = data['smiles_solvent'].tolist()
pka = data['dGsolv_avg [kcal/mol]'].tolist()
sol_solv = [[x,y] for x,y in zip(solute,solvent)]
#preprocess pka too

In [6]:
H_list = []
for x in range(len(sol_solv)):
    if sol_solv[x][0] in ["[H][H]","[2H][2H]","[HH]"]:
        H_list.append(x)
for x in sorted(H_list, reverse = True):
    del sol_solv[x]
    del pka[x]

In [182]:
imp.reload(b)

<module 'basic' from '/Users/u6676643/codes/diympnn/deepchemMPNN/basic.py'>

---
## Training torch models
#### Using the following training sets with 5-fold cross-validation (shuffled)
1. Solute / solvent pairs

---
### Torch models

In [73]:
seed = 24
verbose = False

y_data = torch.Tensor(pka[:1000])
x_data = sol_solv[:1000]
models = ddict(odict)

In [142]:
def generate_score_board(name):
    print(f'{name} CV Scores:')
    for k, v in models[name].cv_scores.items():
         print(f'\t\t- {k}: {np.mean(v):.3f} ± {np.std(v):.3f}')
            
def show_LOSO_scores(name):
    print(f'{name} LOSO scores:')
    for k, v in models[name].LOSO_scores.items():
        v = v[0]
        print(f'\t\t- solvent {k}: MAE: {v[0]}, RMSE: {v[1]}, test size: {v[2]}')
        
def show_LOEO_scores(name):
    print(f'{name} LOEO scores:')
    for k, v in models[name].LOEO_scores.items():
        v = v[0]
        print(f'\t\t- element {k}: MAE: {v[0]}, RMSE: {v[1]}, test size: {v[2]}')

def show_LOMO_scores(name):
    print(f'{name} LOMO scores:')
    for k, v in models[name].LOMO_scores.items():
        v = v[0]
        print(f'\t\t- cutoff mass {k} g/mol: MAE: {v[0]}, RMSE: {v[1]}, test size: {v[2]}')

In [143]:
show_LOMO_scores(name)

MPNN, no interaction LOMO scores:
		- cutoff mass 50 g/mol: MAE: 4.931223392486572, RMSE: 5.829681873321533, test size: 877
		- cutoff mass 100 g/mol: MAE: 5.771317005157471, RMSE: 6.942997455596924, test size: 417
		- cutoff mass 150 g/mol: MAE: 8.06114673614502, RMSE: 9.308589935302734, test size: 134


In [112]:
for k,v in models[name].LOEO_scores.items():
    print(k,v)

F [[3.1151075, 4.830626, 32]]
N [[6.2164173, 7.5845814, 213]]


In [141]:
args = dict(NN_depth=2)
est_cls = c.double_MPNN
name = 'MPNN, no interaction'

cvr = b.CV_torch(est=est_cls, n_folds=2, params=args)
cvr.special_fit(x_data, y_data, 'LOMO')
models[name] = cvr

In [79]:
args = dict(NN_depth=2)
est_cls = c.double_MPNN
name = 'MPNN, no interaction'

models[name] = b.train_cv_model(est_cls, x_data, y_data, params=args, random_state=seed)
generate_score_board(name)

Cross val: 100%|██████████| 10/10 [00:21<00:00,  2.10s/it]

MPNN, no interaction CV Scores:
		- mean_absolute_error: 4.160 ± 0.352
		- rmse: 5.104 ± 0.458
		- r2_score: -1.554 ± 0.331


In [68]:
args = dict(interaction=True)
est_cls = c.double_MPNN
name = 'MPNN, interaction'

models[name] = b.train_cv_model(est_cls, x_data, y_data, params=args, random_state=seed)
generate_score_board(name)

Cross val: 100%|██████████| 5/5 [00:01<00:00,  4.43it/s]

MPNN, interaction CV Scores:
		- mean_absolute_error: 4.377 ± 0.211
		- rmse: 4.463 ± 0.193
		- r2_score: -30.156 ± 11.070


In [69]:
args = dict(interaction=True, atom_messages=False)
est_cls = c.double_MPNN
name = 'D-MPNN, interaction'

models[name] = b.train_cv_model(est_cls, x_data, y_data, params=args, random_state=seed)
generate_score_board(name)

Cross val: 100%|██████████| 5/5 [00:01<00:00,  4.48it/s]

D-MPNN, interaction CV Scores:
		- mean_absolute_error: 4.418 ± 0.278
		- rmse: 4.503 ± 0.259
		- r2_score: -30.916 ± 12.264


In [146]:
args = dict(atom_messages=False)
est_cls = c.double_MPNN
name = 'D-MPNN, no interaction'

models[name] = b.train_cv_model(est_cls, x_data, y_data, params=args, random_state=seed)
generate_score_board(name)

Cross val: 100%|██████████| 10/10 [00:24<00:00,  2.47s/it]

D-MPNN, no interaction CV Scores:
		- mae: 4.080 ± 0.347
		- rmse: 5.036 ± 0.449
		- r2_score: -1.478 ± 0.264


In [151]:
models[name].cv_scores['mae']

[4.8267155,
 3.7570684,
 4.069464,
 4.1323566,
 4.1240726,
 3.7875195,
 4.3714256,
 3.500676,
 3.99001,
 4.238746]

In [189]:
#hyperparameter optimisation testing
from bayes_opt import BayesianOptimization

def fitness(MP_hidden, MP_depth, NN_hidden, NN_depth):
    args = dict(MP_hidden=int(MP_hidden), 
                MP_depth=int(MP_depth), 
                NN_hidden=int(NN_hidden), 
                NN_depth=int(NN_depth))
    results = b.train_cv_model(c.double_MPNN, x_data, y_data, params=args, random_state=seed)
    score = np.mean(results.cv_scores['mae'])
    return -score

# Bounded region of parameter space
pbounds = {'MP_hidden': (100,500), 'MP_depth': (2,4), 'NN_hidden': (100,500), 'NN_depth': (1,3)}

optimizer = BayesianOptimization(
    f=fitness,
    pbounds=pbounds,
    verbose=2,
    random_state=1,
)

In [190]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

|   iter    |  target   | MP_depth  | MP_hidden | NN_depth  | NN_hidden |
-------------------------------------------------------------------------
|  1        | -3.133    |  2.834    |  388.1    |  1.0      |  220.9    |
|  2        | -4.405    |  2.294    |  136.9    |  1.373    |  238.2    |
|  3        | -3.356    |  2.888    |  387.4    |  1.394    |  217.5    |
|  4        | -3.179    |  2.894    |  391.8    |  1.94     |  234.9    |
|  5        | -3.327    |  2.0      |  374.0    |  1.0      |  231.9    |
